# Function Calling, i.e. using Tools

[tools openai tutorial](https://platform.openai.com/docs/guides/function-calling)

A tool can be seen as a fucntion that the model can use/invoke.<br>
Natrually if the modle wants to use a tool in the answer must be present:
- the tool name
- the formatted ouput to fit the paramters of the tool/function

**Attention**: not all model supports tools.

In [1]:
import json
import os

import requests
from openai import OpenAI
from pydantic import BaseModel, Field
from pprint import pprint
client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', 
)

In [2]:
# --------------------------------------------------------------
# Define the tool (function) that we want to call
# --------------------------------------------------------------

def get_weather(latitude, longitude):
    """This is a publically available API that returns the weather for a given location."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]

get_weather(37.7749, -122.4194)

{'time': '2025-03-06T10:15',
 'interval': 900,
 'temperature_2m': 9.0,
 'wind_speed_10m': 18.3}

In [3]:
# --------------------------------------------------------------
# Define schemas for tools
# --------------------------------------------------------------

# schema which 
#   - informs the model what it does 
#   - what input arguments it expects

tools = [ # each dict is a tool
    {
        "type": "function",
        "function": {
            "name": "get_weather", #function name
            "description": "Get current temperature for provided coordinates in celsius.",
                # NB netter is the description, bettwe would be the use of the model of this tool
            "parameters": { # JSON schema defining the function's input arguments
                "type": "object", # function expects an object (i.e., a dictionary) as input.
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                },
                "required": ["latitude", "longitude"], # both latitude and longitude are mandatory.
                "additionalProperties": False, # FORCE: No extra parameters are allowed
            },
            "strict": True, # The model won’t generate extra fields or accept unexpected input.
        },
    }
]

## 1 - Call model with get_weather tool defined

The model will recevive as always an input/prompt.

It will reason and output if it wants to:
- give an asnwer
- call a tool

In [4]:
# ATTENTION: this is conversation history
# this is the beginning, as long as the model calls new tools and thinks NEW messages will be added
messages = [
    {
        "role": "system", # role = specifies who is speaking 
        "content": "You are a helpful weather assistant."
    },
    {
        "role": "user", 
        "content": "What's the weather like in Paris today?"
    },
]

completion = client.chat.completions.create(
    model="llama3.1",
    messages=messages,
    tools=tools, # ATTNETION: tool passed
)

# in completion there will be the answer of the model, it can choose between
# 1) give an asnwer
# 2) call a tool

## 2 - Check what the model decided

In [5]:
pprint(completion.model_dump()) #This prints out the full model response.
# in the completion there will be the answer of the model.
# Within the fields:
# - finish_reason --> what was the esit of the call
# - tool_call
#     - name of function
#     - par to use

# more specific for tool
print("\n")
pprint(completion.choices[0].message.tool_calls)
print("\n")
pprint(completion.choices[0].message)


{'choices': [{'finish_reason': 'tool_calls',
              'index': 0,
              'logprobs': None,
              'message': {'audio': None,
                          'content': '',
                          'function_call': None,
                          'refusal': None,
                          'role': 'assistant',
                          'tool_calls': [{'function': {'arguments': '{"latitude":48.8566,"longitude":2.3522}',
                                                       'name': 'get_weather'},
                                          'id': 'call_z95ioyx6',
                                          'index': 0,
                                          'type': 'function'}]}}],
 'created': 1741256279,
 'id': 'chatcmpl-802',
 'model': 'llama3.1',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_ollama',
 'usage': {'completion_tokens': 29,
           'completion_tokens_details': None,
           'prompt_tokens': 184,
           'prompt_tokens_d

## 3 - Define call-tools function

The model has only answered with the tool and parameters that it wants to use, it hasn't executed any call --> we must do it

In [ ]:
# Define fucntion that depending of model output calls differt tools
def call_function(name, args):
    """
    name: name fo the tool/function to call
    arsg: arguemts to pass
    """
    if name == "get_weather":
        return get_weather(**args) # **args(unpacking the dictionary into keyword arguments).
    if name == "get_gene":
        pass

## 4 - Use the tools

For each tool that the model wants to use, invoke the tool

In [7]:
for tool_call in completion.choices[0].message.tool_calls: # for each tool call

    name = tool_call.function.name # extract the name of the tool
    args = json.loads(tool_call.function.arguments) # extarct the pars

    # appne to message history the answer of the llm
    messages.append(completion.choices[0].message)

    print(name)
    print(args)
    pprint(messages)

    # call the fucntion that calls single tools/fucntions
    result = call_function(name, args)
    print(json.dumps(result)) #converts (or serializes) a Python object into a JSON-formatted STRINGs

    # append result of tool to message history
    # NB The result is formatted, marking it with "role": "tool" and including the tool call ID
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )


get_weather
{'latitude': 48.8566, 'longitude': 2.3522}
[{'content': 'You are a helpful weather assistant.', 'role': 'system'},
 {'content': "What's the weather like in Paris today?", 'role': 'user'},
 ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_z95ioyx6', function=Function(arguments='{"latitude":48.8566,"longitude":2.3522}', name='get_weather'), type='function', index=0)])]
{"time": "2025-03-06T10:15", "interval": 900, "temperature_2m": 10.7, "wind_speed_10m": 9.9}


In [8]:
messages

[{'role': 'system', 'content': 'You are a helpful weather assistant.'},
 {'role': 'user', 'content': "What's the weather like in Paris today?"},
 ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_z95ioyx6', function=Function(arguments='{"latitude":48.8566,"longitude":2.3522}', name='get_weather'), type='function', index=0)]),
 {'role': 'tool',
  'tool_call_id': 'call_z95ioyx6',
  'content': '{"time": "2025-03-06T10:15", "interval": 900, "temperature_2m": 10.7, "wind_speed_10m": 9.9}'}]

## 5 - Rerun the model with new info

Just to be sure, force the ouput of the model to be structured

In [10]:
# Defien pydantic class for stuctred respsonse
class WeatherResponse(BaseModel):
    temperature: float = Field(
            # Field: It does not change the data type, but adds metadata like descriptions, default values, constraints, etc.
        description="The current temperature in celsius for the given location.",
        #default=float("nan"), # default value
        gt=-100, lt=100 #bounds
    )
    response: str = Field(
        description="A natural language response to the user's question."
    )

# Run the model again
completion_2 = client.beta.chat.completions.parse(
    model="llama3.1",
    messages=messages, # ATTNETION: histry is upadated
    tools=tools,
    response_format=WeatherResponse,
)

#Check model response
final_response = completion_2.choices[0].message.parsed
print("formatted output: ", final_response)
print("temp °C: ", final_response.temperature)
print(final_response.response)

formatted output:  temperature=10.7 response="It's currently 10.7°C (49.3°F) in Paris, with a gentle breeze. The wind speed is approximately 9.9 mph."
temp °C:  10.7
It's currently 10.7°C (49.3°F) in Paris, with a gentle breeze. The wind speed is approximately 9.9 mph.
